In [1]:
import os
from pathlib import Path
import pandas as pd


CURRENT_DIR = os.path.abspath('')
DATA_DIR = Path(CURRENT_DIR).parent / "data"
FILE_PATH = DATA_DIR / "sentiment_values.csv"

df = pd.read_csv(FILE_PATH)
df = df.rename(columns = {'hashtag' : 'ss_score'})
df = df.reset_index()
df = df.rename(columns = {'level_0':'hashtag','level_1':'vader_score','level_2':'afinn_score','level_3':'ol_score'})




In [2]:
#Set columns
df.columns=['hashtag','vader_score','afinn_score','ol_score','ss_score','vader_min','vader_max','vader_sum','vader_avg','afinn_min',
            'afinn_max','afinn_sum','afinn_avg','ol_min','ol_max','ol_sum','ol_avg','ss_min','ss_max','ss_sum','ss_avg']

df = df.drop(['vader_min','vader_max','vader_sum','afinn_min','afinn_max','afinn_sum','ol_min','ol_max','ol_sum','ss_min','ss_max','ss_sum'], axis=1)
df.head()

,hashtag,vader_score,afinn_score,ol_score,ss_score,vader_avg,afinn_avg,ol_avg,ss_avg
0,relaxtime,0.8,0.8,2.4,0.8,NaN,0.7375,NaN,NaN
1,melovechilicheese,0.8,0.8,0.8,0.8,NaN,0.9000,1.0,0.8
2,greatmusic,0.8,0.8,2.4,0.8,1.0,0.8875,1.0,0.8
3,rockballad,0.7,0.7,0.7,0.7,NaN,NaN,NaN,NaN
4,amonamarth,0.3,0.3,0.3,0.3,NaN,NaN,0.0,NaN


In [3]:
len(df['hashtag'].unique().tolist())
len(df['hashtag'].unique().tolist())
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5290 entries, 0 to 5289
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   hashtag      5290 non-null   object 
 1   vader_score  3867 non-null   float64
 2   afinn_score  3867 non-null   float64
 3   ol_score     3867 non-null   float64
 4   ss_score     3867 non-null   float64
 5   vader_avg    255 non-null    float64
 6   afinn_avg    2635 non-null   float64
 7   ol_avg       2823 non-null   float64
 8   ss_avg       2160 non-null   float64
dtypes: float64(8), object(1)
memory usage: 372.1+ KB


In [4]:
import numpy as np
df['vader_score'] = df.apply(lambda row: row['vader_avg'] if np.isnan(row['vader_score']) else row[['vader_score']], axis=1)
df['afinn_score'] = df.apply(lambda row: row['afinn_avg'] if np.isnan(row['afinn_score']) else row['afinn_score'], axis=1)
df['ol_score'] = df.apply(lambda row: row['ol_avg'] if np.isnan(row['ol_score']) else row['ol_score'], axis=1)
df['ss_score'] = df.apply(lambda row: row['ss_avg'] if np.isnan(row['ss_score']) else row['ss_score'], axis=1)


In [5]:
df.info()
#vader_score didn't change
#afinn_score increased from 3867 to 4532
#ol_score increased from 3867 to 4831
#ss_score increased from 3867 to 4471

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5290 entries, 0 to 5289
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   hashtag      5290 non-null   object 
 1   vader_score  3871 non-null   float64
 2   afinn_score  4532 non-null   float64
 3   ol_score     4831 non-null   float64
 4   ss_score     4471 non-null   float64
 5   vader_avg    255 non-null    float64
 6   afinn_avg    2635 non-null   float64
 7   ol_avg       2823 non-null   float64
 8   ss_avg       2160 non-null   float64
dtypes: float64(8), object(1)
memory usage: 372.1+ KB


In [6]:
# #Remove all of the unnecessary scores - ol_score has the highest amount of ratings per hashtag
df1 = df.drop(['vader_score','afinn_score','ss_score','vader_avg','afinn_avg','ol_avg','ss_avg'], axis=1)
df1.head(5)

,hashtag,ol_score
0,relaxtime,2.4
1,melovechilicheese,0.8
2,greatmusic,2.4
3,rockballad,0.7
4,amonamarth,0.3


In [7]:
## supprimer les lignes dans lesquelles il y'a au moins un NaN
df1 = df1.dropna(axis = 0, how ='any') 
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4831 entries, 0 to 5289
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   hashtag   4831 non-null   object 
 1   ol_score  4831 non-null   float64
dtypes: float64(1), object(1)
memory usage: 113.2+ KB


In [8]:
df1.sort_values(by='ol_score', ascending=False)

,hashtag,ol_score
4889,thriller,4.8
1287,harmonicalove,4.7
4174,well,3.9
4458,richardmarx,3.6
4924,richmond,3.6
...,...,...
4202,artsohard,0.0
705,indiedisco,0.0
2276,jamofthenight,0.0
4225,justchillin,0.0


In [9]:
#Rename column ol_score 
df1 = df1.rename(columns = {'ol_score' : 'sentiment_score'})
df1.head()

,hashtag,sentiment_score
0,relaxtime,2.4
1,melovechilicheese,0.8
2,greatmusic,2.4
3,rockballad,0.7
4,amonamarth,0.3


In [10]:
count = df1.groupby(['sentiment_score']).count() 
print(count)

                 hashtag
sentiment_score         
0.0                  455
0.1                   42
0.2                  197
0.3                  520
0.4                  191
0.5                   26
0.6                  515
0.6                  106
0.7                  481
0.8                  313
0.9                  287
0.9                   17
0.9                    3
1.0                  522
1.1                   24
1.2                  110
1.2                   24
1.3                   13
1.3                    2
1.4                  268
1.5                   12
1.5                    1
1.6                  251
1.7                    7
1.7                    1
1.8                   48
1.8                   38
1.9                    4
2.0                    4
2.1                  140
2.2                    2
2.4                    5
2.4                    9
2.4                  126
2.6                   16
2.6                    1
2.7                   20
2.8                    4


In [11]:
FILE_PATH = DATA_DIR / 'user_track_hashtag_timestamp.csv'
df2 = pd.read_csv(FILE_PATH)
df2.shape

(17560113, 4)

In [12]:
df2.head()

,user_id,track_id,hashtag,created_at
0,81496937,cd52b3e5b51da29e5893dba82a418a4b,nowplaying,2014-01-01 05:54:21
1,81496937,cd52b3e5b51da29e5893dba82a418a4b,goth,2014-01-01 05:54:21
2,81496937,cd52b3e5b51da29e5893dba82a418a4b,deathrock,2014-01-01 05:54:21
3,81496937,cd52b3e5b51da29e5893dba82a418a4b,postpunk,2014-01-01 05:54:21
4,2205686924,da3110a77b724072b08f231c9d6f7534,NowPlaying,2014-01-01 05:54:22


In [13]:
df2.isna().sum()

user_id       0
track_id      0
hashtag       1
created_at    0
dtype: int64

In [14]:
df2 = df2.dropna(subset=['hashtag'])
df2.isna().sum()

user_id       0
track_id      0
hashtag       0
created_at    0
dtype: int64

In [16]:
# Get the count of the track_id
counts = df2['track_id'].value_counts()

# Drop track_id count < 50
df2 = df2[~df2['track_id'].isin(counts[counts < 50].index)]
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14966614 entries, 0 to 17560112
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   user_id     int64 
 1   track_id    object
 2   hashtag     object
 3   created_at  object
dtypes: int64(1), object(3)
memory usage: 570.9+ MB


In [17]:
df2.user_id.value_counts()

15518784      760846
2293477190    199314
318753478     185412
624797870     141574
1262924646    115376
               ...  
144367888          1
64057544           1
18417673           1
1567289976         1
1600836144         1
Name: user_id, Length: 126887, dtype: int64

In [18]:
df2.track_id.value_counts()

e24deb4963769d25e2c03ad9bf58a028    82551
3d02f9fcad37e6bb227682761039498c    43456
d6b1124fdd64c4b1afa59f967397111d    32735
5758909ef03fc3a2efaa57408ad43f22    32231
acca7456f99b4699871d3fe249e5fe11    32066
                                    ...  
d4ac12c1c07ddf3eb5aeb0ef3fdbf4de       50
a8d66b88b37b575b47e1f8f3bec68cee       50
f3bcbfbc16f3b5aac574216a88434c86       50
a7d341a3d7271f6b42b0c8cbdb909742       50
65c0ae1de781ebf202ba72a3a23b69fb       50
Name: track_id, Length: 49601, dtype: int64

In [20]:
#Merge CSV files into a single file based on hashtag
df_sentiment = pd.merge(df1, df2, on="hashtag", how='inner')
df_sentiment.head()

,hashtag,sentiment_score,user_id,track_id,created_at
0,relaxtime,2.4,104415394,9175ac1532ee7dbe97602866efabac58,2014-05-20 07:24:40
1,relaxtime,2.4,240771401,c2da30eb3450e8a3e5bfa16e8fa527da,2014-10-12 22:46:57
2,relaxtime,2.4,637256774,dcbb5aff8f96a79be9f59bc0e7b5c38d,2014-11-07 13:01:08
3,greatmusic,2.4,252330820,8f2ac86abb8bd48273c8fc95b632e347,2014-02-13 16:18:51
4,greatmusic,2.4,29235188,45bbd6d7cd65dc77596af8c5c0b89a70,2014-05-15 20:38:46


In [21]:
df_sentiment.isna().sum()

hashtag            0
sentiment_score    0
user_id            0
track_id           0
created_at         0
dtype: int64

In [23]:
df_sentiment.shape

(5126717, 5)

In [24]:
df_sentiment['hashtag'].value_counts().head(10)

nowplaying         4870436
kiss92               56437
postpunk             25299
punk                 24730
deathrock            24699
urbantraxxradio      23867
tophits              22836
craveradio            9735
rock                  9483
stonerrock            4297
Name: hashtag, dtype: int64

#### Now that the two CSV files are joined (inner join), the new dataframe df_sentiment is reduced to 5,126,717 rows (from 17,560,114).

In [27]:
FILE_PATH = DATA_DIR / 'context_content_features.csv'
df3 = pd.read_csv(FILE_PATH, usecols=range(0,22))
df3.shape

(11614671, 22)

In [28]:
# Get the count of the track_id
counts = df3['track_id'].value_counts()

# Select the items where the track_id count is less than 50 and remove them
df3 = df3[~df3['track_id'].isin(counts[counts < 50].index)]

df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9143294 entries, 1 to 11614670
Data columns (total 22 columns):
 #   Column            Dtype  
---  ------            -----  
 0   coordinates       object 
 1   instrumentalness  float64
 2   liveness          float64
 3   speechiness       float64
 4   danceability      float64
 5   valence           float64
 6   loudness          float64
 7   tempo             float64
 8   acousticness      float64
 9   energy            float64
 10  mode              float64
 11  key               float64
 12  artist_id         object 
 13  place             object 
 14  geo               object 
 15  tweet_lang        object 
 16  track_id          object 
 17  created_at        object 
 18  lang              object 
 19  time_zone         object 
 20  user_id           float64
 21  id                int64  
dtypes: float64(12), int64(1), object(9)
memory usage: 1.6+ GB


In [29]:
#Drop unnecessary columns before merging with df_sentiment dataframe
df3 = df3.drop(['coordinates','id','place','geo'], axis=1)
df3.head()

,instrumentalness,liveness,speechiness,danceability,valence,loudness,tempo,acousticness,energy,mode,key,artist_id,tweet_lang,track_id,created_at,lang,time_zone,user_id
1,0.017700,0.0638,0.0624,0.769,0.752,-8.252,95.862,0.2670,0.826,1.0,7.0,5cddcd0e314e2f2223ab21937d2c8778,en,da3110a77b724072b08f231c9d6f7534,2014-01-01 05:54:22,en,NaN,2.205687e+09
2,0.000000,0.0860,0.0436,0.675,0.775,-4.432,97.030,0.2170,0.885,0.0,1.0,e41273f43af504714d85465294f1f369,en,ba84d88c10fb0e42d4754a27ead10546,2014-01-01 05:54:22,es,Mountain Time (US & Canada),1.325884e+08
3,0.000000,0.1430,0.0292,0.324,0.333,-5.647,74.101,0.2390,0.574,1.0,7.0,557ce373bd29743eb00a3723ab19ebe8,en,33f95122281f76e7134f9cbea3be980f,2014-01-01 05:54:24,en,Eastern Time (US & Canada),9.767522e+07
5,0.000000,0.1100,0.0375,0.641,0.912,-4.271,93.010,0.0268,0.787,1.0,0.0,f965ec352eb8c0efc0af46244754942f,en,8bd5206b84c968eda0af8bc86d6ab1d1,2014-01-01 05:54:25,en,Central Time (US & Canada),4.522857e+08
6,0.000006,0.3620,0.1340,0.554,0.677,-4.271,126.045,0.0216,0.878,1.0,3.0,a3abd1e016fdba82a91379b7e2b9ab69,en,23ced06ca57d37fa749b1595bc7ed1a4,2014-01-01 05:54:28,en,Quito,6.508628e+07


In [32]:
df3 = df3.dropna()
df3['mode'] = df3['mode'].astype('Int64')
df3.shape
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6413576 entries, 2 to 11614670
Data columns (total 18 columns):
 #   Column            Dtype  
---  ------            -----  
 0   instrumentalness  float64
 1   liveness          float64
 2   speechiness       float64
 3   danceability      float64
 4   valence           float64
 5   loudness          float64
 6   tempo             float64
 7   acousticness      float64
 8   energy            float64
 9   mode              Int64  
 10  key               float64
 11  artist_id         object 
 12  tweet_lang        object 
 13  track_id          object 
 14  created_at        object 
 15  lang              object 
 16  time_zone         object 
 17  user_id           float64
dtypes: Int64(1), float64(11), object(6)
memory usage: 935.8+ MB


In [33]:
df3['lang'].unique()

array(['es', 'en', 'it', 'ja', 'pt', 'nl', 'id', 'en-gb', 'fr', 'ru',
       'de', 'ar', 'ko', 'pl', 'fi', 'ca', 'sv', 'tr', 'gl', 'fa', 'msa',
       'xx-lc', 'cs', 'da', 'zh-tw', 'eu', 'no', 'th', 'el', 'hu',
       'zh-cn', 'uk', 'ro', 'he', 'fil'], dtype=object)

In [49]:
display(df3['lang'].value_counts(normalize=True)*100)
display(df3['tweet_lang'].value_counts(normalize=True)*100)

en       86.235285
ja        4.381943
nl        3.436853
es        1.689543
de        1.351801
fr        0.993773
en-gb     0.611334
ru        0.324003
pt        0.269887
it        0.236252
id        0.209444
tr        0.113834
ko        0.077190
th        0.017026
pl        0.009029
uk        0.008053
fi        0.006967
sv        0.005824
hu        0.003623
ca        0.003539
xx-lc     0.003511
zh-tw     0.002954
cs        0.001979
no        0.001421
fil       0.001087
eu        0.001003
fa        0.000808
zh-cn     0.000585
msa       0.000502
el        0.000390
ar        0.000279
da        0.000167
gl        0.000056
he        0.000028
ro        0.000028
Name: lang, dtype: float64

en     92.163342
es      1.143862
ja      0.831674
in      0.825209
et      0.627302
ht      0.470302
fr      0.461580
de      0.375612
it      0.349111
sk      0.318820
pt      0.312355
pl      0.243636
tl      0.214432
no      0.205292
cy      0.198326
nl      0.184364
ro      0.179237
id      0.167617
fi      0.108484
da      0.105307
zh      0.098898
sv      0.092600
bs      0.053727
lt      0.049965
tr      0.041744
hu      0.038233
sl      0.025999
ko      0.025358
hr      0.023882
is      0.018420
ru      0.017277
vi      0.009781
lv      0.008109
bg      0.003483
und     0.001923
fa      0.001811
th      0.001616
uk      0.000947
ar      0.000334
el      0.000028
Name: tweet_lang, dtype: float64

In [53]:
df3['time_zone'].nunique()
df3['time_zone'].unique()
df3['time_zone'].value_counts(normalize=True)*100

Eastern Time (US & Canada)    29.203842
Central Time (US & Canada)    16.007073
Pacific Time (US & Canada)    15.070398
London                         4.402146
Beijing                        4.363523
                                ...    
PST                            0.000028
Samoa                          0.000028
America/El_Salvador            0.000028
Vladivostok                    0.000028
Europe/Belgrade                0.000028
Name: time_zone, Length: 164, dtype: float64

In [54]:
##Limit dataset to only en (English) language
df3 = df3.loc[~((df3['lang'] != 'en')),:]
df3.info()




<class 'pandas.core.frame.DataFrame'>
Int64Index: 4916702 entries, 3 to 11614670
Data columns (total 18 columns):
 #   Column            Dtype  
---  ------            -----  
 0   instrumentalness  float64
 1   liveness          float64
 2   speechiness       float64
 3   danceability      float64
 4   valence           float64
 5   loudness          float64
 6   tempo             float64
 7   acousticness      float64
 8   energy            float64
 9   mode              Int64  
 10  key               float64
 11  artist_id         object 
 12  tweet_lang        object 
 13  track_id          object 
 14  created_at        object 
 15  lang              object 
 16  time_zone         object 
 17  user_id           float64
dtypes: Int64(1), float64(11), object(6)
memory usage: 717.4+ MB


In [55]:
df3['lang'].unique()

array(['en'], dtype=object)

In [56]:
df4 = df_sentiment.merge(df3, on=['track_id','created_at','user_id'], how='inner')
df4.head()

,hashtag,sentiment_score,user_id,track_id,created_at,instrumentalness,liveness,speechiness,danceability,valence,loudness,tempo,acousticness,energy,mode,key,artist_id,tweet_lang,lang,time_zone
0,greatmusic,2.4,252330820,8f2ac86abb8bd48273c8fc95b632e347,2014-02-13 16:18:51,0.00000,0.1980,0.1690,0.461,0.502,-9.046,143.055,0.1000,0.628,0,4.0,1a17d5f8af99355edd8a92679a02cb0d,en,en,Eastern Time (US & Canada)
1,greatmusic,2.4,29235188,45bbd6d7cd65dc77596af8c5c0b89a70,2014-05-15 20:38:46,0.00001,0.0454,0.0451,0.677,0.886,-13.237,77.837,0.0313,0.544,0,11.0,4a4e2ab094a4521b06252e9fdaf1fd0a,en,en,Eastern Time (US & Canada)
2,nowplaying,0.6,29235188,45bbd6d7cd65dc77596af8c5c0b89a70,2014-05-15 20:38:46,0.00001,0.0454,0.0451,0.677,0.886,-13.237,77.837,0.0313,0.544,0,11.0,4a4e2ab094a4521b06252e9fdaf1fd0a,en,en,Eastern Time (US & Canada)
3,classic,1.0,29235188,45bbd6d7cd65dc77596af8c5c0b89a70,2014-05-15 20:38:46,0.00001,0.0454,0.0451,0.677,0.886,-13.237,77.837,0.0313,0.544,0,11.0,4a4e2ab094a4521b06252e9fdaf1fd0a,en,en,Eastern Time (US & Canada)
4,greatmusic,2.4,195485165,6d45097acaaf6ed5ee55041f53249fa0,2014-10-22 18:36:54,0.00000,0.0769,0.0427,0.591,0.422,-6.315,100.063,0.0171,0.746,1,11.0,f78fb5a7ddce990521f685522f3f8fce,en,en,London


In [57]:
#Convert hashtag info string
df4['hashtag'] = df4['hashtag'].astype(str)

#Convert user_id info string
df4['user_id'] = df4['user_id'].astype(str)

#Show changes
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3094592 entries, 0 to 3094591
Data columns (total 20 columns):
 #   Column            Dtype  
---  ------            -----  
 0   hashtag           object 
 1   sentiment_score   float64
 2   user_id           object 
 3   track_id          object 
 4   created_at        object 
 5   instrumentalness  float64
 6   liveness          float64
 7   speechiness       float64
 8   danceability      float64
 9   valence           float64
 10  loudness          float64
 11  tempo             float64
 12  acousticness      float64
 13  energy            float64
 14  mode              Int64  
 15  key               float64
 16  artist_id         object 
 17  tweet_lang        object 
 18  lang              object 
 19  time_zone         object 
dtypes: Int64(1), float64(11), object(8)
memory usage: 498.8+ MB


In [58]:
#Create new column sentiment that will be the predictor based on the sentiment_score values
df4['sentiment'] = np.where(df4['sentiment_score']>= 0.01, 1, 0)
df4.head()

,hashtag,sentiment_score,user_id,track_id,created_at,instrumentalness,liveness,speechiness,danceability,valence,...,tempo,acousticness,energy,mode,key,artist_id,tweet_lang,lang,time_zone,sentiment
0,greatmusic,2.4,252330820,8f2ac86abb8bd48273c8fc95b632e347,2014-02-13 16:18:51,0.00000,0.1980,0.1690,0.461,0.502,...,143.055,0.1000,0.628,0,4.0,1a17d5f8af99355edd8a92679a02cb0d,en,en,Eastern Time (US & Canada),1
1,greatmusic,2.4,29235188,45bbd6d7cd65dc77596af8c5c0b89a70,2014-05-15 20:38:46,0.00001,0.0454,0.0451,0.677,0.886,...,77.837,0.0313,0.544,0,11.0,4a4e2ab094a4521b06252e9fdaf1fd0a,en,en,Eastern Time (US & Canada),1
2,nowplaying,0.6,29235188,45bbd6d7cd65dc77596af8c5c0b89a70,2014-05-15 20:38:46,0.00001,0.0454,0.0451,0.677,0.886,...,77.837,0.0313,0.544,0,11.0,4a4e2ab094a4521b06252e9fdaf1fd0a,en,en,Eastern Time (US & Canada),1
3,classic,1.0,29235188,45bbd6d7cd65dc77596af8c5c0b89a70,2014-05-15 20:38:46,0.00001,0.0454,0.0451,0.677,0.886,...,77.837,0.0313,0.544,0,11.0,4a4e2ab094a4521b06252e9fdaf1fd0a,en,en,Eastern Time (US & Canada),1
4,greatmusic,2.4,195485165,6d45097acaaf6ed5ee55041f53249fa0,2014-10-22 18:36:54,0.00000,0.0769,0.0427,0.591,0.422,...,100.063,0.0171,0.746,1,11.0,f78fb5a7ddce990521f685522f3f8fce,en,en,London,1


In [61]:
#Drop all null value rows
df4 = df4.dropna()

#Drop unnecessary columns lang and created_at
df4 = df4.drop(['created_at','tweet_lang','lang'], axis=1)

df4.head()

,hashtag,sentiment_score,user_id,track_id,instrumentalness,liveness,speechiness,danceability,valence,loudness,tempo,acousticness,energy,mode,key,artist_id,time_zone,sentiment
0,greatmusic,2.4,252330820,8f2ac86abb8bd48273c8fc95b632e347,0.00000,0.1980,0.1690,0.461,0.502,-9.046,143.055,0.1000,0.628,0,4.0,1a17d5f8af99355edd8a92679a02cb0d,Eastern Time (US & Canada),1
1,greatmusic,2.4,29235188,45bbd6d7cd65dc77596af8c5c0b89a70,0.00001,0.0454,0.0451,0.677,0.886,-13.237,77.837,0.0313,0.544,0,11.0,4a4e2ab094a4521b06252e9fdaf1fd0a,Eastern Time (US & Canada),1
2,nowplaying,0.6,29235188,45bbd6d7cd65dc77596af8c5c0b89a70,0.00001,0.0454,0.0451,0.677,0.886,-13.237,77.837,0.0313,0.544,0,11.0,4a4e2ab094a4521b06252e9fdaf1fd0a,Eastern Time (US & Canada),1
3,classic,1.0,29235188,45bbd6d7cd65dc77596af8c5c0b89a70,0.00001,0.0454,0.0451,0.677,0.886,-13.237,77.837,0.0313,0.544,0,11.0,4a4e2ab094a4521b06252e9fdaf1fd0a,Eastern Time (US & Canada),1
4,greatmusic,2.4,195485165,6d45097acaaf6ed5ee55041f53249fa0,0.00000,0.0769,0.0427,0.591,0.422,-6.315,100.063,0.0171,0.746,1,11.0,f78fb5a7ddce990521f685522f3f8fce,London,1


In [62]:
#Look at unique values for time_zone column
df4.time_zone.unique()

array(['Eastern Time (US & Canada)', 'London', 'Monterrey', 'Alaska',
       'Amsterdam', 'Dublin', 'Central Time (US & Canada)',
       'Pacific Time (US & Canada)', 'Quito', 'Central America',
       'Buenos Aires', 'Mountain Time (US & Canada)', 'Jakarta',
       'Casablanca', 'Mexico City', 'Bangkok', 'Atlantic Time (Canada)',
       'Hawaii', 'Kuala Lumpur', 'Greenland', 'Santiago', 'Madrid',
       'Stockholm', 'Arizona', 'Saskatchewan', 'Beijing', 'Warsaw',
       'Lima', 'Edinburgh', 'Perth', 'Istanbul', 'Lisbon', 'Abu Dhabi',
       'La Paz', 'West Central Africa', 'Athens', 'America/Chicago',
       'Baghdad', 'Singapore', 'Budapest', 'Paris', 'Bern', 'Pretoria',
       'Brisbane', 'Brasilia', 'Moscow', 'Berlin', 'Kyiv', 'Hong Kong',
       'Helsinki', 'New Delhi', 'Sydney', 'Melbourne', 'Harare',
       'Yerevan', 'New Caledonia', 'Adelaide', 'Wellington', 'Copenhagen',
       'Yakutsk', 'Canberra', 'Rome', 'Tokyo', 'Chennai', 'Kathmandu',
       'Seoul', 'Auckland', 'St. Pe

In [63]:
#Make all USA Timezone values consistent
df4['time_zone'].replace('Eastern Time (US & Canada)', 'Eastern Time',inplace=True)
df4['time_zone'].replace('Central Time (US & Canada)', 'Central Time',inplace=True)
df4['time_zone'].replace('Pacific Time (US & Canada)', 'Pacific Time',inplace=True)
df4['time_zone'].replace('Mountain Time (US & Canada)', 'Mountain Time',inplace=True)
df4['time_zone'].replace('Alaska', 'Alaska Time',inplace=True)
df4['time_zone'].replace('Hawaii', 'Hawaii Time',inplace=True)
df4['time_zone'].replace('Arizona', 'Mountain Time',inplace=True)
df4['time_zone'].replace('America/Chicago', 'Central Time',inplace=True)
df4['time_zone'].replace('America/New_York', 'Eastern Time',inplace=True)
df4['time_zone'].replace('America/Los_Angeles', 'Pacific Time',inplace=True)
df4['time_zone'].replace('America/Denver', 'Mountain Time',inplace=True)
df4['time_zone'].replace('America/Detroit', 'Eastern Time',inplace=True)

In [64]:
#Limit dataset to only USA time zones
df4 = df4.loc[~((df4['time_zone'] != 'Eastern Time') & (df4['time_zone'] != 'Central Time') & (df4['time_zone'] != 'Pacific Time') & 
               (df4['time_zone'] != 'Mountain Time') & (df4['time_zone'] != 'Alaska Time') & (df4['time_zone'] != 'Hawaii Time')),:]
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2267492 entries, 0 to 3094591
Data columns (total 18 columns):
 #   Column            Dtype  
---  ------            -----  
 0   hashtag           object 
 1   sentiment_score   float64
 2   user_id           object 
 3   track_id          object 
 4   instrumentalness  float64
 5   liveness          float64
 6   speechiness       float64
 7   danceability      float64
 8   valence           float64
 9   loudness          float64
 10  tempo             float64
 11  acousticness      float64
 12  energy            float64
 13  mode              Int64  
 14  key               float64
 15  artist_id         object 
 16  time_zone         object 
 17  sentiment         int32  
dtypes: Int64(1), float64(11), int32(1), object(5)
memory usage: 322.2+ MB


In [65]:
df4['sentiment'] = df4['sentiment'].astype('Int64')
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2267492 entries, 0 to 3094591
Data columns (total 18 columns):
 #   Column            Dtype  
---  ------            -----  
 0   hashtag           object 
 1   sentiment_score   float64
 2   user_id           object 
 3   track_id          object 
 4   instrumentalness  float64
 5   liveness          float64
 6   speechiness       float64
 7   danceability      float64
 8   valence           float64
 9   loudness          float64
 10  tempo             float64
 11  acousticness      float64
 12  energy            float64
 13  mode              Int64  
 14  key               float64
 15  artist_id         object 
 16  time_zone         object 
 17  sentiment         Int64  
dtypes: Int64(2), float64(11), object(5)
memory usage: 333.0+ MB


In [66]:
## DESTINATION_PATH = DATA_DIR.parent / 'cleaned_dataset.csv'
## df4.to_csv(DESTINATION_PATH)

## artist_id and "hashtag were not dropped" so 5 dtypes object instead of 3
#Reduced the dataset from 4,916,702 to 2,267,492 by limiting it to USA Timezone data only.
## Create MVP Dataset minimum viable product


In [67]:
# Reorder columns
df_mvp = df4[['sentiment','sentiment_score', 'hashtag', 'user_id','track_id','artist_id','time_zone','instrumentalness',
              'liveness','speechiness','danceability','valence','loudness','tempo','acousticness','energy','mode','key']]
df_mvp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2267492 entries, 0 to 3094591
Data columns (total 18 columns):
 #   Column            Dtype  
---  ------            -----  
 0   sentiment         Int64  
 1   sentiment_score   float64
 2   hashtag           object 
 3   user_id           object 
 4   track_id          object 
 5   artist_id         object 
 6   time_zone         object 
 7   instrumentalness  float64
 8   liveness          float64
 9   speechiness       float64
 10  danceability      float64
 11  valence           float64
 12  loudness          float64
 13  tempo             float64
 14  acousticness      float64
 15  energy            float64
 16  mode              Int64  
 17  key               float64
dtypes: Int64(2), float64(11), object(5)
memory usage: 333.0+ MB


In [68]:
# Create new dataset with only user_id, track_id and time_zone and the category variables
df_timezone = df_mvp.drop(['user_id','track_id', 'hashtag', 'artist_id', 'user_id','sentiment','sentiment_score','instrumentalness','liveness','speechiness',
                             'danceability','valence','loudness','tempo','acousticness','energy','mode','key'], axis=1)
df_timezone.head()

,time_zone
0,Eastern Time
1,Eastern Time
2,Eastern Time
3,Eastern Time
7,Alaska Time


In [71]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# One Hot Encode the category data set
one_hot_df = pd.get_dummies(df_timezone)
one_hot_df.head()

,time_zone_Alaska Time,time_zone_Central Time,time_zone_Eastern Time,time_zone_Hawaii Time,time_zone_Mountain Time,time_zone_Pacific Time
0,0,0,1,0,0,0
1,0,0,1,0,0,0
2,0,0,1,0,0,0
3,0,0,1,0,0,0
7,1,0,0,0,0,0


In [72]:
# Rename the columns
one_hot_df.columns = ['tz_Alaska_Time','tz_Central_Time','tz_Eastern_Time','tz_Hawaii_Time','tz_Mountain_Time','tz_Pacific_Time']
one_hot_df.head()

,tz_Alaska_Time,tz_Central_Time,tz_Eastern_Time,tz_Hawaii_Time,tz_Mountain_Time,tz_Pacific_Time
0,0,0,1,0,0,0
1,0,0,1,0,0,0
2,0,0,1,0,0,0
3,0,0,1,0,0,0
7,1,0,0,0,0,0


In [73]:
#Drop time_zone from MVP dataset
df_mvp = df_mvp.drop(['time_zone'], axis=1)
df_mvp.columns

Index(['sentiment', 'sentiment_score', 'hashtag', 'user_id', 'track_id',
       'artist_id', 'instrumentalness', 'liveness', 'speechiness',
       'danceability', 'valence', 'loudness', 'tempo', 'acousticness',
       'energy', 'mode', 'key'],
      dtype='object')

In [74]:
#Concatenate one hot encoded dataframe
df_mvp = pd.concat([df_mvp,one_hot_df], axis=1)
df_mvp.head()

,sentiment,sentiment_score,hashtag,user_id,track_id,artist_id,instrumentalness,liveness,speechiness,danceability,...,acousticness,energy,mode,key,tz_Alaska_Time,tz_Central_Time,tz_Eastern_Time,tz_Hawaii_Time,tz_Mountain_Time,tz_Pacific_Time
0,1,2.4,greatmusic,252330820,8f2ac86abb8bd48273c8fc95b632e347,1a17d5f8af99355edd8a92679a02cb0d,0.000000,0.1980,0.1690,0.461,...,0.100000,0.628,0,4.0,0,0,1,0,0,0
1,1,2.4,greatmusic,29235188,45bbd6d7cd65dc77596af8c5c0b89a70,4a4e2ab094a4521b06252e9fdaf1fd0a,0.000010,0.0454,0.0451,0.677,...,0.031300,0.544,0,11.0,0,0,1,0,0,0
2,1,0.6,nowplaying,29235188,45bbd6d7cd65dc77596af8c5c0b89a70,4a4e2ab094a4521b06252e9fdaf1fd0a,0.000010,0.0454,0.0451,0.677,...,0.031300,0.544,0,11.0,0,0,1,0,0,0
3,1,1.0,classic,29235188,45bbd6d7cd65dc77596af8c5c0b89a70,4a4e2ab094a4521b06252e9fdaf1fd0a,0.000010,0.0454,0.0451,0.677,...,0.031300,0.544,0,11.0,0,0,1,0,0,0
7,1,0.3,amonamarth,124052165,d53b2de022666067050dae8f6645edc2,942c9f2520684c22eb6216a92b711f9e,0.000142,0.1130,0.0709,0.355,...,0.000004,0.980,0,7.0,1,0,0,0,0,0


In [75]:
DESTINATION_PATH = DATA_DIR / 'cleaned_dataset.csv'
df_mvp.to_csv(DESTINATION_PATH)

## La suite n'est pas à utiliser pour l'instant

In [11]:
## Tentative de merge avec le dataframe MuSe.csv 
## à ne pas utiliser pour l'instant
import os
from pathlib import Path
import pandas as pd

CURRENT_DIR = os.path.abspath('')
DATA_DIR = Path(CURRENT_DIR).parent / "data"
FILE_PATH = DATA_DIR/'muse_v3.csv'

df6 = pd.read_csv(FILE_PATH)
df6

,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre
0,https://www.last.fm/music/eminem/_/%2527till%2...,'Till I Collapse,Eminem,['aggressive'],6,4.550000,5.273125,5.690625,cab93def-26c5-4fb0-bedd-26ec4c1619e1,4xkOaSrkexMciUUogZKVTS,rap
1,https://www.last.fm/music/metallica/_/st.%2banger,St. Anger,Metallica,['aggressive'],8,3.710000,5.833000,5.427250,727a2529-7ee8-4860-aef6-7959884895cb,3fOc9x06lKJBhz435mInlH,metal
2,https://www.last.fm/music/rick%2bross/_/speedi...,Speedin',Rick Ross,['aggressive'],1,3.080000,5.870000,5.490000,NaN,3Y96xd4Ce0J47dcalLrEC8,rap
3,https://www.last.fm/music/m.i.a./_/bamboo%2bbanga,Bamboo Banga,M.I.A.,"['aggressive', 'fun', 'sexy', 'energetic']",13,6.555071,5.537214,5.691357,99dd2c8c-e7c1-413e-8ea4-4497a00ffa18,6tqFC1DIOphJkCwrjVzPmg,hip-hop
4,https://www.last.fm/music/dope/_/die%2bmf%2bdie,Die MF Die,Dope,['aggressive'],7,3.771176,5.348235,5.441765,b9eb3484-5e0e-4690-ab5a-ca91937032a5,5bU4KX47KqtDKKaLM4QCzh,metal
...,...,...,...,...,...,...,...,...,...,...,...
89996,https://www.last.fm/music/gaby%2bhoffmann%2b%2...,Battle,Gaby Hoffmann & Jay Duplass,['transparent'],1,5.370000,3.450000,5.330000,NaN,NaN,NaN
89997,https://www.last.fm/music/omar%2brodriguez-lop...,Hands Tied to the Roots of a Hemorrhage,Omar Rodriguez-Lopez,['transparent'],4,5.797887,4.132254,5.570563,NaN,NaN,progressive rock
89998,https://www.last.fm/music/ace%2bwhite/_/crookf...,Crookfield Zoo,Ace White,['transparent'],1,5.370000,3.450000,5.330000,NaN,NaN,NaN
89999,https://www.last.fm/music/second%2bchyld/_/tra...,Transparent (Full Version),Second Chyld,['transparent'],1,5.370000,3.450000,5.330000,NaN,NaN,NaN


In [12]:
df6['mbid'].isna().sum()

28784

In [4]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90001 entries, 0 to 90000
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   lastfm_url              90001 non-null  object 
 1   track                   90001 non-null  object 
 2   artist                  90001 non-null  object 
 3   seeds                   90001 non-null  object 
 4   number_of_emotion_tags  90001 non-null  int64  
 5   valence_tags            90001 non-null  float64
 6   arousal_tags            90001 non-null  float64
 7   dominance_tags          90001 non-null  float64
 8   mbid                    61217 non-null  object 
 9   spotify_id              61630 non-null  object 
 10  genre                   83362 non-null  object 
dtypes: float64(3), int64(1), object(7)
memory usage: 7.6+ MB


In [13]:

df6 = df6.dropna(axis=0, subset='mbid')
df6.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61217 entries, 0 to 90000
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   lastfm_url              61217 non-null  object 
 1   track                   61217 non-null  object 
 2   artist                  61217 non-null  object 
 3   seeds                   61217 non-null  object 
 4   number_of_emotion_tags  61217 non-null  int64  
 5   valence_tags            61217 non-null  float64
 6   arousal_tags            61217 non-null  float64
 7   dominance_tags          61217 non-null  float64
 8   mbid                    61217 non-null  object 
 9   spotify_id              50118 non-null  object 
 10  genre                   58559 non-null  object 
dtypes: float64(3), int64(1), object(7)
memory usage: 5.6+ MB


In [15]:
df6 = df6.drop(['lastfm_url', 'track', 'artist', 'spotify_id', 'genre'], axis=1)
df6.info()
df6.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 61217 entries, 0 to 90000
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   seeds                   61217 non-null  object 
 1   number_of_emotion_tags  61217 non-null  int64  
 2   valence_tags            61217 non-null  float64
 3   arousal_tags            61217 non-null  float64
 4   dominance_tags          61217 non-null  float64
 5   mbid                    61217 non-null  object 
dtypes: float64(3), int64(1), object(2)
memory usage: 5.3+ MB


seeds                     0
number_of_emotion_tags    0
valence_tags              0
arousal_tags              0
dominance_tags            0
mbid                      0
dtype: int64

In [17]:
#df6.duplicated().sum()
#print(df6['track'].duplicated().sum())
#print(df6['spotify_id'].duplicated().sum())
print(df6['mbid'].duplicated().sum())


0


In [54]:
df6['track_id'] = df6['mbid'].apply(lambda x: x[:8] + x[9:13] + x[14:18] + x[19:23] +x[24:])


df6

,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,track_id
0,['aggressive'],6,4.550000,5.273125,5.690625,cab93def-26c5-4fb0-bedd-26ec4c1619e1,cab93def26c54fb0bedd26ec4c1619e1
1,['aggressive'],8,3.710000,5.833000,5.427250,727a2529-7ee8-4860-aef6-7959884895cb,727a25297ee84860aef67959884895cb
3,"['aggressive', 'fun', 'sexy', 'energetic']",13,6.555071,5.537214,5.691357,99dd2c8c-e7c1-413e-8ea4-4497a00ffa18,99dd2c8ce7c1413e8ea44497a00ffa18
4,['aggressive'],7,3.771176,5.348235,5.441765,b9eb3484-5e0e-4690-ab5a-ca91937032a5,b9eb34845e0e4690ab5aca91937032a5
5,['aggressive'],9,2.971389,5.537500,4.726389,49e7b4d2-3772-4301-ba25-3cc46ceb342e,49e7b4d237724301ba253cc46ceb342e
...,...,...,...,...,...,...,...
89985,['transparent'],1,5.370000,3.450000,5.330000,ffd10ae8-858a-4b5e-819e-64f1174bbd42,ffd10ae8858a4b5e819e64f1174bbd42
89986,['transparent'],2,6.685000,4.405000,5.625000,16c3d394-c4d4-4dc2-bbf1-b2bef3ac861c,16c3d394c4d44dc2bbf1b2bef3ac861c
89991,['transparent'],6,4.301667,3.341667,4.466667,b744fdbc-7b4c-49a1-a3d6-99047d82eb34,b744fdbc7b4c49a1a3d699047d82eb34
89994,['transparent'],2,5.760000,3.875000,5.005000,da36a5a0-0f6b-4d67-8429-c567d8bbf215,da36a5a00f6b4d678429c567d8bbf215


In [64]:
test_list = pd.Series(df6['track_id']).tolist()
print(test_list)

['cab93def26c54fb0bedd26ec4c1619e1', '727a25297ee84860aef67959884895cb', '99dd2c8ce7c1413e8ea44497a00ffa18', 'b9eb34845e0e4690ab5aca91937032a5', '49e7b4d237724301ba253cc46ceb342e', '1a8260835585445fa708415dc90aa050', '4435982cb83e4daaaf2b9f3430036bb7', 'fe1cc051faa74953b331f6196cd3ddae', '585398ed127545799451e8dd7db9d59c', 'a3c325cefac442b985dab3c9e0f243af', '3bc2c1a943bc45b287fc4313eb2534fe', 'f6cc8417e590494caba9e3cbc536442d', '8050e7e72c7a4390bf77eaedf293c626', '23b462863b27431b89439be9d37836f9', 'b83fe0b7d40d4e779b2b606d36e4a391', 'bf2761fe8b4d4cc8bdea0ccda7c3fed1', '1b49361ae594490caa60dde81b235d24', '9e387cb2843f4db0a0eb8ba677dfd93d', '74fa2e8f01c94eb38078402c877b7e3b', '4f7635bba53e4772835182177499cf20', 'cb6d5d3633e74e7ab233fd9f610cafe2', '20408c001e6d4f0ba90724a6aae9b33a', '5a69aa31a0044f5a90c77389565431e9', '88af084eac2540b484e3cd0e488200cc', '43ec7c8e6b8a41ec8348d141572b8b78', '5512523ae1eb4712bd63e30495783993', 'ccf93ad3ed7346c3999391633ee45aa8', '5cb8474a86034ff98a3a77043e

In [71]:
#print('b9eb34845e0e4690ab5aca91937032a5' in df6['track_id'])
#df6['track_id'].str.contains('45bbd6d7cd65dc77596af8c5c0b89a70').sum()
#print(len(df6['track_id'][0]))

32


In [76]:
test_list = []
for i in df6['track_id']:
    test_list.append(i)
print(len(test_list))    
print(test_list[0])

61217
cab93def26c54fb0bedd26ec4c1619e1


In [ ]:
FILE_PATH = DATA_DIR/'muse_v3.csv'

df6 = pd.read_csv(FILE_PATH)
df6 = df6.dropna(axis=0, subset='mbid')
df6 = df6.drop(['lastfm_url', 'track', 'artist', 'spotify_id', 'genre'], axis=1)
df6['track_id'] = df6['mbid'].apply(lambda x: x[:8] + x[9:13] + x[14:18] + x[19:23] +x[24:])

In [ ]:
test_list = []
for i in df6['track_id']:
    test_list.append(i)



In [ ]:
df_mvp['track_id'].isin(test_list).sum()